In [1]:
import turicreate
import re, string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [2]:
personality_model = turicreate.load_model("personality_model/")

In [3]:
stemmer = SnowballStemmer("english")

In [7]:
def clean_data(text):
    text = text.lower()
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())
    text = re.sub(r'http\S+', '', text)
    chars = re.escape(string.punctuation)
    text = re.sub(r'['+chars+']', '', text)
    words = text.split()
    clean_text = ""
    for word in words:
        clean_text += stemmer.stem(word) + " ";
    return clean_text

In [5]:
def predict(text):
    text = clean_data(text)
    with open("testset.csv", "w") as fp:
        fp.write("type,text\n")
        fp.write("default,"+text)
    sf = turicreate.SFrame("testset.csv")
    sf["tfidf"] = turicreate.text_analytics.count_words(sf["text"])
    result = personality_model.predict(sf[0])
    return result